In [1]:
import matplotlib.pyplot as plt
from IPython import display

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Categorical
from tqdm.notebook import tqdm
from gym.wrappers import StepAPICompatibility
from tqdm import tqdm

In [2]:
seed = 2023
def fix(env, seed):
    env.reset(seed=seed)
    env.action_space.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
      torch.cuda.manual_seed_all(seed)

In [3]:
import gym
import random
env = gym.make('LunarLander-v2')
# env = StepAPICompatibility(env, new_step_api=True)
fix(env, seed)

In [29]:
class PolicyGradientNetwork(nn.Module):

    def __init__(self, hidden_dim=16):
        super().__init__()
        self.fc1 = nn.Linear(8, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, 4)

    def forward(self, state):
        hid = torch.tanh(self.fc1(state))
        hid = torch.tanh(hid)
        return F.softmax(self.fc3(hid), dim=-1)

# Value Network
class ValueNet(nn.Module):
    def __init__(self, hidden_dim=16):
        super().__init__()

        self.hidden = nn.Linear(8, hidden_dim)
        self.output = nn.Linear(hidden_dim, 4)

    def forward(self, s):
        outs = self.hidden(s)
        outs = F.relu(outs)
        value = self.output(outs)
        return value
    
from torch.optim.lr_scheduler import StepLR
class PolicyGradientAgent():

    def __init__(self, network):
        self.network = network
        self.optimizer = optim.SGD(self.network.parameters(), lr=0.002)
        self.scheduler = StepLR(self.optimizer, step_size=1000, gamma=0.9)
        self.rewards = None
        self.discounted_rewards = None

    def forward(self, state):
        return self.network(state)

    def learn(self, log_probs, rewards):
        loss = (-log_probs * rewards).sum() # You don't need to revise this to pass simple baseline (but you can)

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

    def learn_A2C(self, log_probs, rewards, states, value_func):
        with torch.no_grad():
            values = value_func(states).squeeze()
        advantages = rewards - values
        loss = (-log_probs * advantages).sum()

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

    def sample(self, state):
        # Convert state to a FloatTensor
        if isinstance(state, np.ndarray):
            state = torch.from_numpy(state).float()  # Convert directly from NumPy array to tensor
        elif isinstance(state, list):
            state = torch.FloatTensor(state)
        else:
            raise ValueError(f"Unexpected state format: {type(state)}")

        # Get action probabilities and sample an action
        action_prob = self.network(state)
        action_dist = Categorical(action_prob)
        action = action_dist.sample()
        log_prob = action_dist.log_prob(action)
        return action.item(), log_prob


    # def sample(self, state):
    #     action_prob = self.network(torch.FloatTensor(state))
    #     action_dist = Categorical(action_prob)
    #     action = action_dist.sample()
    #     log_prob = action_dist.log_prob(action)
    #     return action.item(), log_prob

In [30]:
network = PolicyGradientNetwork()
agent = PolicyGradientAgent(network)

agent.network.train()  # Switch network into training mode
EPISODE_PER_BATCH = 5  # update the agent every 5 episodes
NUM_BATCH = 500        # totally update the agent for 500 episodes
gamma = 0.99            # Discount factor

avg_total_rewards, avg_final_rewards = [], []

prg_bar = tqdm(range(NUM_BATCH))
for batch in prg_bar:

    log_probs, rewards = [], []
    total_rewards, final_rewards = [], []

    # Update the training loop to extract the state correctly
    for episode in range(EPISODE_PER_BATCH):
        result = env.reset()
        if isinstance(result, tuple):
            state, _ = result  # If reset returns a tuple, extract the state
        else:
            state = result  # If it returns only the state

        total_reward, total_step = 0, 0
        episode_rewards = []  # Store episode-specific rewards

        while True:
            action, log_prob = agent.sample(state)  # Get action and log probability
            next_state, reward, done, truncated, _ = env.step(action)

            log_probs.append(log_prob)  # Store log probability
            episode_rewards.append(reward)  # Store immediate reward
            state = next_state
            total_reward += reward
            total_step += 1

            if done or truncated:
                final_rewards.append(reward)
                total_rewards.append(total_reward)
                break


        # Convert episode rewards to discounted cumulative rewards
        discounted_rewards = []
        cumulative_reward = 0

        # Compute cumulative decaying rewards for the episode
        for r in reversed(episode_rewards):
            cumulative_reward = r + gamma * cumulative_reward
            discounted_rewards.insert(0, cumulative_reward)  # Insert at the front to keep the right order

        rewards.extend(discounted_rewards)  # Extend the rewards list with the episode's discounted rewards

    print(f"Rewards shape: {np.shape(rewards)}")

    # Record training process
    avg_total_reward = sum(total_rewards) / len(total_rewards)
    avg_final_reward = sum(final_rewards) / len(final_rewards)
    avg_total_rewards.append(avg_total_reward)
    avg_final_rewards.append(avg_final_reward)
    prg_bar.set_description(f"Total: {avg_total_reward: 4.1f}, Final: {avg_final_reward: 4.1f}")

    # Update the agent using cumulative decaying rewards
    rewards = np.array(rewards)
    rewards = (rewards - np.mean(rewards)) / (np.std(rewards) + 1e-9)  # Normalize the rewards
    agent.learn(torch.stack(log_probs), torch.from_numpy(rewards))

    # Print shapes for debugging
    print("Log probs size: ", torch.stack(log_probs).size())
    print("Rewards size: ", torch.from_numpy(rewards).size())

    # Print the AVG total reward achieved so far
    print(f"AVG Total Reward so far: {avg_total_reward:.2f}")


Total: -101.9, Final: -100.0:   0%|          | 1/500 [00:00<02:33,  3.26it/s]

Rewards shape: (437,)
Log probs size:  torch.Size([437])
Rewards size:  torch.Size([437])
AVG Total Reward so far: -101.91


Total: -373.0, Final: -100.0:   0%|          | 2/500 [00:00<02:31,  3.29it/s]

Rewards shape: (543,)
Log probs size:  torch.Size([543])
Rewards size:  torch.Size([543])
AVG Total Reward so far: -372.96


Total: -179.3, Final: -100.0:   1%|          | 3/500 [00:00<02:36,  3.17it/s]

...

Total: -273.3, Final: -100.0:  14%|█▍        | 70/500 [00:23<02:43,  2.63it/s]

Rewards shape: (766,)
Log probs size:  torch.Size([766])
Rewards size:  torch.Size([766])
AVG Total Reward so far: -273.30


Total: -161.0, Final: -100.0:  14%|█▍        | 70/500 [00:23<02:26,  2.93it/s]

Rewards shape: (580,)


KeyboardInterrupt: 

In [12]:
env.step(action)

(array([-0.01233473,  1.3889593 , -0.41315836, -0.31314465,  0.01414089,
         0.09668748,  0.        ,  0.        ], dtype=float32),
 0.35363735044923034,
 False,
 False,
 {})

In [11]:
action

2

In [17]:
network = PolicyGradientNetwork()
agent = PolicyGradientAgent(network)

agent.network.train()  # Switch network into training mode
EPISODE_PER_BATCH = 5  # update the agent every 5 episodes
NUM_BATCH = 500        # totally update the agent for 500 episodes
gamma = 0.99           # Discount factor

avg_total_rewards, avg_final_rewards = [], []

prg_bar = tqdm(range(NUM_BATCH))
for batch in prg_bar:
    log_probs, rewards = [], []
    total_rewards, final_rewards = [], []

    # Collect trajectory
    for episode in range(EPISODE_PER_BATCH):
        state = env.reset()
        total_reward, total_step = 0, 0
        episode_rewards = []  # Store episode-specific rewards

        while True:
            action, log_prob = agent.sample(state)  # Get action and log probability
            next_state, reward, done, truncated, _ = env.step(action)

            log_probs.append(log_prob)  # Store log probability
            episode_rewards.append(reward)  # Store immediate reward
            state = next_state
            total_reward += reward
            total_step += 1

            if done or truncated:
                final_rewards.append(reward)
                total_rewards.append(total_reward)
                break

        # Convert episode rewards to discounted cumulative rewards
        discounted_rewards = []
        cumulative_reward = 0

        # Compute cumulative decaying rewards for the episode
        for r in reversed(episode_rewards):
            cumulative_reward = r + gamma * cumulative_reward
            discounted_rewards.insert(0, cumulative_reward)  # Insert at the front to keep the right order

        rewards.extend(discounted_rewards)  # Extend the rewards list with the episode's discounted rewards

    # Record training process
    avg_total_reward = sum(total_rewards) / len(total_rewards)
    avg_final_reward = sum(final_rewards) / len(final_rewards)
    avg_total_rewards.append(avg_total_reward)
    avg_final_rewards.append(avg_final_reward)
    prg_bar.set_description(f"Total: {avg_total_reward: 4.1f}, Final: {avg_final_reward: 4.1f}")

    # Update the agent using cumulative decaying rewards
    rewards = np.array(rewards)
    rewards = (rewards - np.mean(rewards)) / (np.std(rewards) + 1e-9)  # Normalize the rewards
    agent.learn(torch.stack(log_probs), torch.from_numpy(rewards).float())

    # Print shapes less frequently to avoid slowing down training
    if batch % 10 == 0:
        print(f"Batch {batch}: Log probs size: {torch.stack(log_probs).size()}, Rewards size: {torch.from_numpy(rewards).size()}")
        print(f"AVG Total Reward so far: {avg_total_reward:.2f}")

plt.plot(avg_total_rewards)
plt.xlabel('Batch')
plt.ylabel('Average Total Reward')
plt.title('Policy Gradient Training Progress')
plt.show()


  0%|          | 0/500 [00:00<?, ?it/s]


ValueError: Unexpected state format: <class 'tuple'>